# Numeric Metric
> Base class for all numeric metrics


In [ ]:
#| default_exp metric.numeric

In [ ]:
#| export

import typing as t
from dataclasses import dataclass, field
from pydantic import BaseModel, create_model
from ragas_experimental.metric import Metric, MetricResult
from ragas_experimental.metric.decorator import create_metric_decorator

@dataclass
class NumericMetric(Metric):
    range: t.Tuple[float,float]
    
    def _get_response_model(self, with_reasoning: bool) -> t.Type[BaseModel]:
        """Get or create a response model based on reasoning parameter."""
        
        if with_reasoning in self._response_models:
            return self._response_models[with_reasoning]
        
        model_name = 'response_model'
        fields = {"result": (float,...)}
        
        if with_reasoning:
            fields["reason"] = (str, ...) #type: ignore
        
        model = create_model(model_name, **fields)
        self._response_models[with_reasoning] = model
        return model 

    def _ensemble(self,results:t.List[MetricResult]) -> MetricResult:

        if len(results)==1:
            return results[0]
    
        candidates = [candidate.result for candidate in results]
        result = sum(candidates)/len(candidates)
        reason = results[0].reason
    
        return MetricResult(result=result,reason=reason)
    
    
numeric_metric = create_metric_decorator(NumericMetric)


/opt/homebrew/Caskroom/miniforge/base/envs/random/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Example usage

In [ ]:

#| eval: false

from ragas_experimental.llm import ragas_llm
from openai import OpenAI

llm = ragas_llm(provider="openai",model="gpt-4o",client=OpenAI())


my_metric = NumericMetric(
    name='helpfulness',
    llm=llm,
    prompt="Evaluate if given answer is helpful\n\n{response}",
    range=(0,10),
)

result = my_metric.score(response="this is my response")
result #gives "low"
result.reason #gives reasoning from llm



"The provided input lacks context or content to determine if it is helpful as it merely states 'this is my response' without any additional information."

### Write custom numeric metric

In [ ]:

#| eval: false
from ragas_experimental.metric import MetricResult

@numeric_metric(llm=llm,
    prompt="Evaluate if given answer is helpful\n\n{response}",
    name='new_metric',range=(0,10))
def my_metric(llm,prompt,**kwargs):

        class response_model(BaseModel):
             output: int
             reason: str
        
        traces = {}
        traces['input'] = kwargs
        response = llm.generate(prompt.format(**kwargs),response_model=response_model)
        traces['output'] = response.dict()
        total = response.output
        if total < 1:
            score = 0
        else:
            score = 10
        return MetricResult(result=score,reason=response.reason,traces=traces)

result = my_metric.score(response='my response') # result
result # 10
result.reason # the reason for the answer

result1 = my_metric.score(response='my response 1') # result
result2 = my_metric.score(response='my response 2') # result

result1 + result2 # should be addable and behave like a float


10